In [1]:
# Import Libraries
import cv2
import numpy as np
import os
import re

In [2]:
# Define label-to-name mapping
label_to_name = {
    1: "Aaron_Eckhart",  # Update with actual names
    2: "Aaron_Guiel",
    3: "Aaron_Patterson",
    4: "Aaron_Peirsol",
    5: "Aaron_Pena",
    6: "Aaron_Sorkin",
    7: "Aaron_Tippin",
    8: "Abbas_Kiarostami",
    9: "Abba_Eban", 
    10: "Abdel_Aziz_Al-Hakim",
    18: "Abdullah_Gul",
    # Add more mappings as needed
}

Face Detection

In [3]:
# Helper 
# Function to detect faces in an image
def detect_faces(image_path):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return img, gray, faces

Prepare Training Data

In [4]:
# Function to prepare training data
def prepare_training_data(data_folder_path, image_size=(100, 100)):
    dirs = os.listdir(data_folder_path)
    faces = []
    labels = []
    label_map = {}
    next_label = 1

    for dir_name in dirs:
        # Check if the directory name is an integer
        try:
            label = int(dir_name)
        except ValueError:
            # If not, create a mapping for non-integer labels
            if dir_name not in label_map:
                label_map[dir_name] = next_label
                next_label += 1
            label = label_map[dir_name]

        subject_dir_path = os.path.join(data_folder_path, dir_name)
        subject_images_names = os.listdir(subject_dir_path)

        for image_name in subject_images_names:
            image_path = os.path.join(subject_dir_path, image_name)
            img, gray, detected_faces = detect_faces(image_path)
            if len(detected_faces) == 0:
                continue
            (x, y, w, h) = detected_faces[0]
            face_img = gray[y:y+h, x:x+w]
            
            # Resize face images to a consistent size
            face_img = cv2.resize(face_img, image_size)
            faces.append(face_img)
            labels.append(label)

    # Convert lists to NumPy arrays
    faces = np.array(faces)
    labels = np.array(labels, dtype=np.int32)
    
    return faces, labels

In [5]:
# Prepare training data
faces, labels = prepare_training_data(r'C:\Users\alesa\OneDrive\Desktop\Dataset')

Train Face Recognizer

In [6]:
# Train the Face Recognizer
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(faces, np.array(labels))

In [7]:
# Function to predict faces in test images
def predict(test_img_paths):
    results = []
    
    for test_img_path in test_img_paths:
        img, gray, detected_faces = detect_faces(test_img_path)
        
        if len(detected_faces) == 0:
            print(f"No faces detected in image: {test_img_path}")
            results.append(img)  # Append original image if no face is detected
            continue
        
        for (x, y, w, h) in detected_faces:
            roi_gray = gray[y:y+w, x:x+h]
            roi_gray = cv2.resize(roi_gray, (100, 100))  # Resize to match training image size
            label, confidence = face_recognizer.predict(roi_gray)
            name = label_to_name.get(label, "Unknown")
            print(f"Image: {test_img_path} - Predicted label: {label} ({name}) with confidence: {confidence}")
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(img, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        
        # Collect result for each image
        results.append(img)

    return results


In [8]:
# Function to concatenate images in a grid format
def concatenate_images_in_grid(images, images_per_row=6):
    if len(images) == 0:
        return np.zeros((100, 100, 3), dtype=np.uint8)  # Return a blank image if no images

    # Ensure all images have the same height
    heights = [img.shape[0] for img in images]
    max_height = max(heights)

    # Resize all images to have the same height
    resized_images = []
    for img in images:
        if img.shape[0] < max_height:
            # Resize image to have the same height
            new_width = int(img.shape[1] * (max_height / img.shape[0]))
            resized_img = cv2.resize(img, (new_width, max_height))
        else:
            resized_img = img
        resized_images.append(resized_img)

    # Create rows of images with consistent width
    rows = []
    for i in range(0, len(resized_images), images_per_row):
        row_images = resized_images[i:i+images_per_row]
        max_width = max(img.shape[1] for img in row_images)
        padded_row_images = [cv2.copyMakeBorder(img, 0, 0, 0, max_width - img.shape[1], cv2.BORDER_CONSTANT, value=[0, 0, 0]) for img in row_images]
        row_image = np.hstack(padded_row_images)
        rows.append(row_image)

    # Pad rows to ensure same width for concatenation
    max_row_width = max(row.shape[1] for row in rows)
    padded_rows = [cv2.copyMakeBorder(row, 0, 0, 0, max_row_width - row.shape[1], cv2.BORDER_CONSTANT, value=[0, 0, 0]) for row in rows]

    # Concatenate rows vertically
    concatenated_image = np.vstack(padded_rows)
    return concatenated_image


In [9]:
# List of test image paths
test_img_paths = [
r"C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0007.jpg",
r"C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0001.jpg",
r"C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0002.jpg",
r"C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0003.jpg",
r"C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0017.jpg",
r"C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0018.jpg",
r"C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0019.jpg",
   # Add more paths as needed
]

In [10]:
# Predict faces in all test images
predicted_imgs = predict(test_img_paths)

Image: C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0007.jpg - Predicted label: 18 (Abdullah_Gul) with confidence: 0.0
Image: C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0001.jpg - Predicted label: 18 (Abdullah_Gul) with confidence: 0.0
Image: C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0002.jpg - Predicted label: 18 (Abdullah_Gul) with confidence: 0.0
Image: C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0003.jpg - Predicted label: 18 (Abdullah_Gul) with confidence: 0.0
Image: C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0017.jpg - Predicted label: 18 (Abdullah_Gul) with confidence: 0.0
Image: C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0018.jpg - Predicted label: 18 (Abdullah_Gul) with confidence: 0.0
Image: C:\Users\alesa\OneDrive\Desktop\Dataset\Abdullah_Gul\Abdullah_Gul_0019.jpg - Predicted label: 18 (Abdullah_Gul) with confidence: 0.0


In [11]:
# Concatenate images in a grid format
concatenated_img = concatenate_images_in_grid(predicted_imgs, images_per_row=6)

In [ ]:
# Display concatenated image
cv2.imshow('Concatenated Images', concatenated_img)
cv2.waitKey(0)
cv2.destroyAllWindows()